In [ ]:
from datetime import datetime

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import randint
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import sys
import os
sys.path.append(os.path.abspath('..'))
from build_dataset import data_cleaning

In [ ]:
df = pd.read_csv('../data/The_Dataset_v1-1.csv', parse_dates = ["Date"])
df = data_cleaning.clean_dataset(df)
df = data_cleaning.remove_systems(df)
df = data_cleaning.weather_code_to_category(df)
df = df.drop(columns=['Date', 'System ID'])
df.describe()

In [ ]:
# Split into testing and training data:
X = df.drop('Efficiency (kWh/kW)', axis=1)
y = df['Efficiency (kWh/kW)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Start with a linear regression with forward subset selection.
lin_reg = LinearRegression()
sfs = SequentialFeatureSelector(lin_reg, direction="forward", cv=5, n_jobs=-1)
sfs.fit(X_train, y_train)

# Predict with selected features
X_train_sfs = sfs.transform(X_train)
X_test_sfs = sfs.transform(X_test)
lin_reg.fit(X_train_sfs, y_train)
y_pred_sfs = lin_reg.predict(X_test_sfs)
rmse_sfs = root_mean_squared_error(y_test, y_pred_sfs)
r2_sfs = r2_score(y_test, y_pred_sfs)

# Print the results
print("Forward Subset Selection Linear Regression:")
print(f"RMSE: {rmse_sfs:.4f}")
print(f"R²:   {r2_sfs:.4f}")


# Also try an Elastic Net
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("elastic", ElasticNet(max_iter=10000))
])

param_dist = {
    "elastic__alpha": np.logspace(-3, 1, 100),
    "elastic__l1_ratio": np.linspace(0, 1, 100)
}

random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_dist,
    n_iter=20,
    scoring="neg_root_mean_squared_error",
    cv=5,
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train, y_train)
y_pred_en = random_search.predict(X_test)
rmse_en = root_mean_squared_error(y_test, y_pred_en)
r2_en = r2_score(y_test, y_pred_en)

print("\nElastic Net Regression (Random Search):")
print(f"RMSE: {rmse_en:.4f}")
print(f"R²:   {r2_en:.4f}")
print(f"Best Params: {random_search.best_params_}")

In [ ]:
# The elastic net shows more lasso regression than ridge regression (l1_ratio is closer to 1 than 0).
# RMSE is 0.87 in both cases. This has units of hours (efficiency is daily energy production (kWh) / power rating (kW)),
# so it indicates an average prediction error of 0.87 * 60 = 52 minutes.

# Let's see which features were selected by forward subset selection
subset_coefs = sfs.get_support().astype(int)

# View the coefficients for the elastic net:
best_pipeline = random_search.best_estimator_
elastic_model = best_pipeline.named_steps["elastic"]
elastic_coefs = elastic_model.coef_
intercept = elastic_model.intercept_

coef_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Elastic Coefficient": elastic_coefs,
    "Selected by FSS?": subset_coefs
})

# Sort by the elastic net coefficient
coef_df["|Elastic Coefficient|"] = coef_df["Elastic Coefficient"].abs()
coef_df = coef_df.sort_values(by="|Elastic Coefficient|", ascending=False).drop(columns="|Elastic Coefficient|")

coef_df

In [ ]:
# shortwave_radiation_sum has a much higher coefficient than all other variables.
# Try a random forest.

# There will be multiple random forests so let's write a function:
def run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 20):
    # Time it
    current_time = datetime.now().strftime("%H:%M:%S")
    print("Start time:", current_time)
        
    # Initialize model
    rf = RandomForestRegressor(random_state=42, n_jobs=1)
    
    # Randomized search
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=5,
        scoring='neg_root_mean_squared_error',
        verbose=2,
        n_jobs=8,
        random_state=42,
        return_train_score=True
    )
    
    # Run the search
    random_search.fit(X_train, y_train)
    best_rf = random_search.best_estimator_
    
    # Test set evaluation
    y_pred = best_rf.predict(X_test)
    rmse = root_mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print("Best RF Test RMSE:", rmse)
    print("Best RF Test R²:", r2)
    
    # Show top CV results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results = cv_results.sort_values(by='mean_test_score', ascending=False)
    print("\nCV Results:")
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    display(cv_results[['mean_test_score', 'std_test_score', 'params']])
    current_time = datetime.now().strftime("%H-%M-%S")
    cv_results.to_csv(f'random_forest_cv_results_{current_time}.csv')
    
    # Print end time
    current_time = datetime.now().strftime("%H:%M:%S")
    print("End time:", current_time)
    
    return best_rf, random_search


param_dist = {
        'n_estimators': randint(100, 500),
        'max_depth': randint(10, 50),
        'min_samples_leaf': [1, 2, 4, 8],
        'max_features': ['sqrt', None]
    }

# Run it with 20 iterations (takes over an hour).
# Commented out to prevent accidental execution.
# Run a small random forest so that code in the following cells works, although comments interpret results from the full search.
# Uncomment the final line of the cell and run for the full random forest search.
best_rf, random_search = run_rf_search(X_train, y_train, X_test, y_test, {
        'n_estimators': [100],
        'max_depth': [10],
        'min_samples_leaf': [2],
        'max_features': ['sqrt']
    }, n_iter = 1)
# best_rf, random_search = run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 20)

In [ ]:
# A small improvement in RMSE. The linear regression gave 52 minutes, and the random forest gives 48 minutes (RMSE is 0.8 hours).
# Changing hyperparameters has little effect on the performance.

# View the importances of the features:
importances = pd.Series(best_rf.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance")
plt.show()

In [ ]:
# It is likely that there is little improvement to be made on 0.8 RMSE without:
# collecting more data
# Improving the quality of data: e.g. some systems on pvoutput.org record the weather condition and temperature.
#     So a really high quality dataset would only include these systems, and only include those days where the pvoutput weather matches the historical forecast.
# Taking into account the specific hardware used in each solar panel system,
#     because that will be causing some variance in solar panel output given the same weather conditions
# Only including those days where the solar panel system was operational for the entire duration of daylight

# given my time constraints, I will try the following:
# 1. add elevation above sea level to the dataset.
# 2. try engineering some new variables
# 3. try removing the weather category dummy variables, not because it would improve the model, but because they are unimportant and would make the model simpler

# Load dataset with elevation variable
df = pd.read_csv('../data/The_Dataset_v1-2.csv', parse_dates = ["Date"])
df = data_cleaning.clean_dataset(df)
df = data_cleaning.remove_systems(df)
df = data_cleaning.weather_code_to_category(df)
df = df.drop(columns=['Date', 'System ID'])

# Plot efficiency vs elevation
# Control for weather by filtering for some weather codes:
df_clear = df[(df.filter(like='weather_category') == 0).all(axis=1)]
df_partly_cloudy = df[df['weather_category_partly_cloudy'] == 1]
df_overcast = df[df['weather_category_overcast'] == 1]

# Plot the filtered data
plt.scatter(df_clear['Elevation (m)'], df_clear['Efficiency (kWh/kW)'])
plt.title('Clear')
plt.xlabel('Elevation')
plt.ylabel('Efficiency (kWh/kW)')
plt.show()

# Plot 2: Partly Cloudy
plt.scatter(df_partly_cloudy['Elevation (m)'], df_partly_cloudy['Efficiency (kWh/kW)'], color='orange')
plt.title('Partly Cloudy')
plt.xlabel('Elevation')
plt.ylabel('Efficiency (kWh/kW)')
plt.show()

# Plot 3: Overcast
plt.scatter(df_overcast['Elevation (m)'], df_overcast['Efficiency (kWh/kW)'], color='gray')
plt.title('Overcast')
plt.xlabel('Elevation')
plt.ylabel('Efficiency (kWh/kW)')
plt.show()

In [ ]:
# No obvious correlation between elevation and efficiency, though a random forest might still show an improvement.

# Start by doing a quick test using the already trained random forest:
X = df.drop('Efficiency (kWh/kW)', axis=1)
y = df['Efficiency (kWh/kW)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_rf = RandomForestRegressor(**best_rf.get_params())
best_rf.fit(X_train, y_train)

# Test set evaluation
y_pred = best_rf.predict(X_test)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Best RF Test RMSE:", rmse)
print("Best RF Test R²:", r2)

In [ ]:
# A minor improvement, even using an untuned random forest model.
# Let's see if that improvement is robust by running a new random search (with fewer hyperparameter combinations):
best_rf_e, random_search_e = run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 5)

In [ ]:
# The RMSE has decreased again after running the search again. Clearly, elevation is an important variable, and
# the importances plot should confirm this:
importances = pd.Series(best_rf_e.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance")
plt.show()

In [ ]:
# Elevation is the second most important feature.
# Strangely, "shortwave_radiation_sum" now has far greater relative importance compared to the other features.
# This can be explained by looking at the hyperparameters of the two random forests trained so far:
print(best_rf.get_params())  # random forest on the dataset without elevation
print(best_rf_e.get_params())  # with elevation

In [ ]:
# Where the first random forest has 'max_features': 'sqrt', the new random forest has 'max_features': 'None'.
# This means that each decision tree in the new random forest was allowed to check through every variable to find the optimum split,
# so it was free to choose 'shortwave_radiation_sum' for every split (as opposed to sqrt(n) of the splits where there are n variables).

# Let's run a few more random forests with different available features.
# To reiterate: I will try engineering some new variables, and removing the weather category dummy variables.

# Use a slightly smaller parameter distribution:
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': randint(10, 30),
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt']  # To more clearly see the relative importances of features
}

df_new = df.copy()
# Create some potentially useful new features
df_new['radiation_strength'] = df_new['shortwave_radiation_sum'] / df_new['daylight_duration']
df_new['sunshine_ratio'] = df_new['sunshine_duration'] / df_new['daylight_duration']
df_new['precipitation_intensity'] = df_new['precipitation_sum'] / df_new['precipitation_hours']

# transform wind direction
wind_rad = np.deg2rad(df_new['wind_direction_10m_dominant'])
df_new['wind_dir_sin'] = np.sin(wind_rad)
df_new['wind_dir_cos'] = np.cos(wind_rad)

In [ ]:
# The code from here takes about half an hour to run. It is not necessary to run this code for the remaining cells to run.

# Training the random forests
# without weather category
X = df.drop(columns=[col for col in df.columns if col.startswith('weather_category_')] + ['Efficiency (kWh/kW)'])
y = df['Efficiency (kWh/kW)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_rf_no_code, random_search_no_code = run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 10)

importances = pd.Series(best_rf_no_code.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance")
plt.show()

# with new variables
X = df_new.drop('Efficiency (kWh/kW)', axis=1)
y = df_new['Efficiency (kWh/kW)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_rf_new, random_search_new = run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 10)

importances = pd.Series(best_rf_new.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance")
plt.show()

# with new variables without weather category
X = df_new.drop(columns=[col for col in df.columns if col.startswith('weather_category_')] + ['Efficiency (kWh/kW)'])
y = df_new['Efficiency (kWh/kW)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_rf_new_no_code, random_search_new_no_code = run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 10)

importances = pd.Series(best_rf_new_no_code.feature_importances_, index=X.columns)
importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance")
plt.show()

In [ ]:
# None of the models show any significant improvement in predictive power. Stick with the original dataset plus elevation.

# Now is a good time for an overnight hyperparameter search.
param_dist = {
        'n_estimators': randint(100, 500),
        'max_depth': randint(20, 50),
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', None]
    }

X = df.drop('Efficiency (kWh/kW)', axis=1)
y = df['Efficiency (kWh/kW)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Commented out to prevent accidental execution. It takes several hours to run.
'''
best_rf, search = run_rf_search(X_train, y_train, X_test, y_test, param_dist, n_iter = 200)

joblib.dump(best_rf, 'rf_big_search.pkl')
joblib.dump(search, 'big_search.pkl')
'''
# Instead I will load in the results from a csv:
hyperparam_search_results = pd.read_csv("large_hyperparam_search.csv")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
display(hyperparam_search_results[['mean_test_score', 'std_test_score', 'params']])

In [ ]:
# The highest performing models have large values for n_estimators and max_depth.
# However, the highest perfoming model has a very large file size after saving it with joblib.
# This will be an issue for deploying the model in my web app.

# The performance drops off very slowly and there is a negligible difference between the highest-performing models
# and models with max_features = None, min_samples_leaf = 2, and much smaller max_depth and n_estimators.
# So let's train a smaller forest and presume that it will perform nearly as well as the "best" model, but with a much smaller file size.

params = {
        'n_estimators': [100],
        'max_depth': [20],
        'min_samples_leaf': [2],
        'max_features': [None]
    }
small_rf, small_search = run_rf_search(X_train, y_train, X_test, y_test, param_dist = params, n_iter = 1)

In [ ]:
# RMSE is 0.0048 hours (17.28 seconds) greater on this much smaller forest,
# a negligible loss of accuracy for the huge reduction in file size and memory usage.

joblib.dump(small_rf, 'rf_100.pkl')